# Dog breeds
## Using Fast.ai's Deep Learning techniques to validate the highest accuracy in the classification of dog breeds

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
#import sys
#sys.path.append("../fastai") 

In [ ]:
from fastai.imports import *
from fastai.torch_imports import *
from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *

In [ ]:
PATH = "data/"
sz=224
arch=resnext101_64
bs=58

In [ ]:
label_csv = f'{PATH}labels.csv'
n = len(list(open(label_csv)))-1
val_idx = get_cv_idxs(n)

## Data Exploration after unzipping

In [ ]:
!pwd
!ls {PATH}

In [ ]:
label_df= pd.read_csv(label_csv)

In [ ]:
label_df.head()

In [ ]:
label_df.pivot_table(index='breed',aggfunc=len).sort_values('id',ascending=False).head()

In [ ]:
transformations = tfms_from_model(arch,sz,aug_tfms=transforms_side_on,max_zoom=1.2)

In [ ]:
data = ImageClassifierData.from_csv(PATH,'train', f'{PATH}labels.csv',test_name='test',val_idxs=val_idx,suffix='.jpg',tfms=transformations,bs=bs)

In [ ]:
fn=PATH+data.trn_ds.fnames[0]; fn

In [ ]:
n

In [ ]:
val_idx

In [ ]:
len(val_idx)

In [ ]:
img = PIL.Image.open(fn); img

In [ ]:
img.size

In [ ]:
size_d = {k:PIL.Image.open(PATH+k).size for k in data.trn_ds.fnames}

In [ ]:
row_sz,col_sz = list(zip(*size_d.values()))

In [ ]:
row_sz = np.array(row_sz);col_sz= np.array(col_sz)

In [ ]:
row_sz[:5]

In [ ]:
plt.hist(row_sz);

In [ ]:
plt.hist(row_sz[row_sz<1000]);

# Initial model

In [ ]:
def get_data(sz,bs):
    tfms = tfms_from_model(arch,sz,aug_tfms=transforms_side_on,max_zoom=1.2)
    data = ImageClassifierData.from_csv(PATH,'train',f'{PATH}labels.csv',test_name='test',num_workers=4, 
                                        val_idxs=val_idx,suffix='.jpg',tfms=tfms,bs=bs)
    return data if sz >300 else data.resize(340,'tmp')

## Precompute

In [ ]:
data = get_data(sz,bs)

In [ ]:
learn = ConvLearner.pretrained(arch,data,precompute=True)

In [ ]:
learn.fit(1e-2,5)

## Augment

In [ ]:
from sklearn import metrics

In [ ]:
data = get_data(sz,bs)

In [ ]:
learn = ConvLearner.pretrained(arch,data,precompute=True,ps=0.5)

In [ ]:
learn.fit(1e-2,2)

In [ ]:
learn.precompute=False

In [ ]:
learn.fit(1e-2, 5, cycle_len=1)

In [ ]:
learn.fit(1e-2, 3, cycle_len=1, cycle_mult=2)

In [ ]:
learn.save('224_pre')

In [ ]:
learn.load('224_pre')

## Increase size

In [ ]:
learn.set_data(get_data(299,bs))
learn.freeze()

In [ ]:
learn.fit(1e-2, 3, cycle_len=1)

In [ ]:
learn.fit(1e-2, 3, cycle_len=1, cycle_mult=2)

In [ ]:
log_preds,y = learn.TTA()
probs = np.mean(np.exp(log_preds), axis=0)

In [ ]:
accuracy_np(probs,y), metrics.log_loss(y, probs)

In [ ]:
learn.save('299_pre')

In [ ]:
learn.load('299_pre')

In [ ]:
learn.fit(1e-2,1,cycle_len=2)

In [ ]:
log_preds,y = learn.TTA(is_test=True)
probs = np.exp(log_preds)
probs.shape

In [ ]:
probs[0,:,:].shape

In [ ]:
data.classes

In [ ]:
df = pd.DataFrame(probs[0,:,:], columns = data.classes)

In [ ]:
df.columns

In [ ]:
df.insert(0,'id',[o[5:-4] for o in data.test_ds.fnames])

In [ ]:
df.columns

In [ ]:
df.head()

In [ ]:
SUBM = f'{PATH}subm/'
os.makedirs(SUBM,exist_ok=True)
df.to_csv(f'{SUBM}subm.gz', compression='gzip',index=False)

In [ ]:
FileLink(f'{SUBM}subm.gz')